## **Crawling**

### Detik

In [ ]:
# import requests
# from datetime import datetime, timedelta
# from bs4 import BeautifulSoup
# import os

# # --- Konfigurasi ---
# os.makedirs("../data", exist_ok=True)
# URL_OUTPUT_FILE = "../data/crawled_urls.txt"
# LOG_FILE = "../data/crawl_logs.txt"
# PORTAL_NAME = "Detik-Inet"

# def log_message(message):
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, "a", encoding="utf-8") as f:
#         f.write(log_entry)

# def extract_links_from_html(html):
#     """Ambil semua link artikel game dari halaman indeks Detik."""
#     soup = BeautifulSoup(html, "html.parser")
#     anchors = soup.select("article.list-content__item a.media__link")
    
#     urls = []
#     for a in anchors:
#         href = a.get("href")
#         if href and "/games-news/" in href:
#             urls.append(f"{PORTAL_NAME};{href.strip()}")
            
#     return list(set(urls))

# def crawl_detik(start_date_str="2025-01-01", end_date_str="2025-11-17"):
#     log_message(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
    
#     base_url = "https://inet.detik.com/games-news/indeks?date="
#     start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
#     end_date = datetime.strptime(end_date_str, "%Y-%m-%d")

#     total_days = (end_date - start_date).days + 1
#     day_counter = 1
#     all_urls_found = []

#     current_date = start_date
#     while current_date <= end_date:
#         date_for_log = current_date.strftime('%Y-%m-%d')
#         log_message(f"({day_counter}/{total_days}) Memproses tanggal: {date_for_log}")

#         date_for_url = current_date.strftime("%m/%d/%Y")
#         url = base_url + date_for_url

#         try:
#             response = requests.get(url, timeout=15)
#             response.raise_for_status()
#             page_urls = extract_links_from_html(response.text)
            
#             if page_urls:
#                 all_urls_found.extend(page_urls)
#                 log_message(f"  -> Ditemukan {len(page_urls)} link.")
#             else:
#                 log_message(f"  -> Tidak ada artikel ditemukan.")
        
#         except requests.exceptions.RequestException as e:
#             log_message(f"  -> ERROR: {e}")
        
#         current_date += timedelta(days=1)
#         day_counter += 1

#     return all_urls_found

# if __name__ == "__main__":
#     detik_urls = crawl_detik(start_date_str="2025-01-01", end_date_str="2025-11-17")

#     if detik_urls:
#         unique_urls = sorted(list(set(detik_urls)))
#         log_message(f"Menyimpan total {len(unique_urls)} link unik yang ditemukan...")
#         with open(URL_OUTPUT_FILE, "a", encoding="utf-8") as f:
#             for url in unique_urls:
#                 f.write(url + "\n")
#         log_message("Penyimpanan link berhasil.")
#     else:
#         log_message("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
#     log_message(f"===== Proses Crawling Selesai =====\n")

[2025-11-19 10:41:30] ===== Memulai Proses Crawling Detik-Inet =====
[2025-11-19 10:41:30] (1/321) Memproses tanggal: 2025-01-01
[2025-11-19 10:41:31]   -> Ditemukan 2 link.
[2025-11-19 10:41:31] (2/321) Memproses tanggal: 2025-01-02
[2025-11-19 10:41:32]   -> Ditemukan 3 link.
[2025-11-19 10:41:32] (3/321) Memproses tanggal: 2025-01-03
[2025-11-19 10:41:32]   -> Ditemukan 2 link.
[2025-11-19 10:41:32] (4/321) Memproses tanggal: 2025-01-04
[2025-11-19 10:41:33]   -> Ditemukan 2 link.
[2025-11-19 10:41:33] (5/321) Memproses tanggal: 2025-01-05


KeyboardInterrupt: 

### Gamebrott

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# from datetime import datetime
# import os
# import re 

# from bs4 import XMLParsedAsHTMLWarning
# import warnings

# warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

# # --- Konfigurasi ---
# SITEMAP_INDEX_URL = "https://gamebrott.com/sitemap_index.xml"
# URL_OUTPUT_FILE = "../data/crawled_urls.txt" 
# LOG_FILE = "../data/crawl_logs.txt"
# PORTAL_NAME = "Gamebrott"

# output_dir = os.path.dirname(URL_OUTPUT_FILE)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def log_to_file(message):
#     """Fungsi untuk menulis pesan log ke file dengan timestamp."""
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, 'a', encoding='utf-8') as f:
#         f.write(log_entry)

# def get_sitemap_number(url):
#     """Mengekstrak nomor dari URL sitemap untuk sorting yang benar."""
#     match = re.search(r'post-sitemap(\d+)\.xml', url)
#     return int(match.group(1)) if match else 0

# def crawl_xml_sitemap():
#     """
#     Fungsi utama untuk crawling sitemap XML, dengan logging dan penyimpanan append.
#     """
#     log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
    
#     newly_found_urls = []
    
#     try:
#         log_to_file(f"Mengambil sitemap index dari: {SITEMAP_INDEX_URL}")
#         response_main = requests.get(SITEMAP_INDEX_URL, headers=HEADERS, timeout=15)
#         response_main.raise_for_status()

#         # Gunakan "lxml-xml" atau "xml" dengan lxml sebagai parser
#         # Jika masih error, gunakan "html.parser" sebagai fallback
#         try:
#             soup_main = BeautifulSoup(response_main.content, "lxml-xml")
#         except:
#             log_to_file("  -> Mencoba parser alternatif...")
#             soup_main = BeautifulSoup(response_main.content, "html.parser")
        
#         all_sitemap_links = [loc.text for loc in soup_main.find_all('loc')]
        
#         if not all_sitemap_links:
#             log_to_file("ERROR: Tidak ada tag <loc> yang ditemukan di sitemap index.")
#             return

#         log_to_file(f"Total sitemap ditemukan di index: {len(all_sitemap_links)}")
        
#         post_sitemap_links = [url for url in all_sitemap_links if 'post-sitemap' in url]
#         log_to_file(f"Menyaring sitemap... Ditemukan {len(post_sitemap_links)} link yang mengandung 'post-sitemap'.")

#         post_sitemap_links.sort(key=get_sitemap_number)
        
#         sitemaps_to_process = post_sitemap_links[:3]
#         log_to_file(f"Membatasi proses hanya untuk {len(sitemaps_to_process)} sitemap pertama.")
        
#         for i, sitemap_url in enumerate(sitemaps_to_process, 1):
#             log_to_file(f"({i}/{len(sitemaps_to_process)}) Memproses sitemap: {sitemap_url}")
#             try:
#                 response_post = requests.get(sitemap_url, headers=HEADERS, timeout=15)
#                 response_post.raise_for_status()

#                 # Gunakan parser yang sama
#                 try:
#                     soup_post = BeautifulSoup(response_post.content, "lxml-xml")
#                 except:
#                     soup_post = BeautifulSoup(response_post.content, "html.parser")
                
#                 article_links_in_page = []
#                 url_blocks = soup_post.find_all('url')
                
#                 for block in url_blocks:
#                     loc_tag = block.find('loc')
#                     if loc_tag:
#                         article_links_in_page.append(loc_tag.text)
                
#                 log_to_file(f"  -> Ditemukan {len(article_links_in_page)} link artikel (gambar diabaikan).")
#                 newly_found_urls.extend(article_links_in_page)

#             except requests.exceptions.RequestException as e:
#                 log_to_file(f"  -> ERROR: Gagal mengambil atau memproses {sitemap_url}: {e}")
#                 continue

#     except requests.exceptions.RequestException as e:
#         log_to_file(f"FATAL ERROR: Gagal mengambil sitemap index utama. Proses dihentikan. Error: {e}")
#         return

#     if newly_found_urls:
#         log_to_file(f"Menyimpan {len(newly_found_urls)} link baru ke file {URL_OUTPUT_FILE}...")
#         try:
#             with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
#                 for url in newly_found_urls:
#                     # Menggunakan variabel PORTAL_NAME
#                     f.write(f"{PORTAL_NAME};{url}\n")
#             log_to_file("Penyimpanan link berhasil.")
#         except IOError as e:
#             log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
#     else:
#         log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
#     log_to_file(f"Total link yang didapat pada sesi ini: {len(newly_found_urls)}")
#     log_to_file("===== Proses Crawling Selesai =====\n")


# if __name__ == "__main__":
#     crawl_xml_sitemap()

[2025-11-17 19:11:32] ===== Memulai Proses Crawling Gamebrott =====
[2025-11-17 19:11:32] Mengambil sitemap index dari: https://gamebrott.com/sitemap_index.xml
[2025-11-17 19:11:33]   -> Mencoba parser alternatif...
[2025-11-17 19:11:33] Total sitemap ditemukan di index: 243
[2025-11-17 19:11:33] Menyaring sitemap... Ditemukan 217 link yang mengandung 'post-sitemap'.
[2025-11-17 19:11:33] Membatasi proses hanya untuk 3 sitemap pertama.
[2025-11-17 19:11:33] (1/3) Memproses sitemap: https://gamebrott.com/post-sitemap1.xml
[2025-11-17 19:11:34]   -> Ditemukan 201 link artikel (gambar diabaikan).
[2025-11-17 19:11:34] (2/3) Memproses sitemap: https://gamebrott.com/post-sitemap2.xml
[2025-11-17 19:11:36]   -> Ditemukan 200 link artikel (gambar diabaikan).
[2025-11-17 19:11:36] (3/3) Memproses sitemap: https://gamebrott.com/post-sitemap3.xml
[2025-11-17 19:11:37]   -> Ditemukan 200 link artikel (gambar diabaikan).
[2025-11-17 19:11:37] Menyimpan 601 link baru ke file ../data/crawled_urls.tx

### Kotakgames

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# from datetime import datetime
# import os

# import urllib3
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# START_YEAR = 2025
# END_YEAR = 2025  # Sama dengan START_YEAR untuk hanya mengambil tahun 2025
# MAX_LINKS = 500

# # --- Konfigurasi ---
# URL_OUTPUT_FILE = "../data/crawled_urls.txt"
# LOG_FILE = "../data/crawl_logs.txt"
# PORTAL_NAME = "Kotakgame"

# output_dir = os.path.dirname(URL_OUTPUT_FILE)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def log_to_file(message):
#     # Gunakan format yang kompatibel dengan Windows dan Linux
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, 'a', encoding='utf-8') as f:
#         f.write(log_entry)

# def get_last_page(year_base_url):
#     """Fungsi ini sekarang menerima base URL untuk tahun tertentu."""
#     first_page_url = year_base_url + "1/"
#     try:
#         log_to_file(f"  -> Mencari halaman terakhir dari: {first_page_url}")
#         response = requests.get(first_page_url, headers=HEADERS, timeout=15, verify=False)
#         response.raise_for_status()
        
#         soup = BeautifulSoup(response.content, "html.parser")
#         last_page_link = soup.find('a', class_='prevnext', text='LAST »')
        
#         if last_page_link and last_page_link.has_attr('href'):
#             href = last_page_link['href']
#             page_number = int(href.strip('/').split('/')[-1])
#             log_to_file(f"  -> Halaman terakhir ditemukan: {page_number}")
#             return page_number
#         else:
#             log_to_file("  -> WARNING: Link 'LAST »' tidak ditemukan. Mengasumsikan hanya ada 1 halaman.")
#             return 1
            
#     except requests.exceptions.RequestException as e:
#         log_to_file(f"  -> ERROR: Tidak bisa mengakses halaman pertama untuk tahun ini. Error: {e}")
#         return 0 # Mengembalikan 0 jika gagal, untuk dilewati
#     except (ValueError, IndexError):
#         log_to_file("  -> FATAL ERROR: Gagal mem-parsing nomor halaman terakhir dari link.")
#         return 0

# def crawl_kotakgame_multi_year(max_links=None):
#     """
#     Crawl artikel dari Kotakgame.com
    
#     Args:
#         max_links (int, optional): Jumlah maksimal link yang ingin diambil.
#                                    Jika None, akan mengambil semua link.
#     """
#     log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} (Tahun {START_YEAR}-{END_YEAR}) =====")
#     if max_links:
#         log_to_file(f"Target: Mengambil maksimal {max_links} link")
    
#     grand_total_urls = []
    
#     for year in range(START_YEAR, END_YEAR + 1):
#         # Cek apakah sudah mencapai limit
#         if max_links and len(grand_total_urls) >= max_links:
#             log_to_file(f"Sudah mencapai limit {max_links} link. Menghentikan crawling.")
#             break
            
#         log_to_file(f"--- Memproses Tahun {year} ---")
        
#         # Base URL sekarang dinamis berdasarkan tahun
#         base_url_for_year = f"https://www.kotakgame.com/berita/index/{year}/0/0/"
        
#         last_page = get_last_page(base_url_for_year)
#         if last_page == 0:
#             log_to_file(f"Melewati tahun {year} karena gagal mendapatkan info halaman.")
#             continue # Lanjut ke tahun berikutnya

#         urls_this_year = []
#         for page_num in range(1, last_page + 1):
#             # Cek limit sebelum memproses halaman baru
#             if max_links and len(grand_total_urls) >= max_links:
#                 log_to_file(f"    -> Sudah mencapai limit {max_links} link. Melewati halaman berikutnya.")
#                 break
                
#             page_url = f"{base_url_for_year}{page_num}/"
#             log_to_file(f"    ({page_num}/{last_page}) Memproses halaman: {page_url}")
            
#             try:
#                 response = requests.get(page_url, headers=HEADERS, timeout=15, verify=False)
#                 response.raise_for_status()
                
#                 soup = BeautifulSoup(response.content, "html.parser")
#                 article_links_found = soup.select('div#contenta div.detailfeature h4 a')
                
#                 page_urls = []
#                 for link_tag in article_links_found:
#                     if link_tag.has_attr('href'):
#                         relative_url = link_tag['href']
#                         full_url = f"https://www.kotakgame.com{relative_url}"
#                         page_urls.append(full_url)
                
#                 # Batasi jumlah URL yang diambil jika sudah mendekati limit
#                 if max_links:
#                     remaining_links = max_links - len(grand_total_urls)
#                     if remaining_links < len(page_urls):
#                         page_urls = page_urls[:remaining_links]
#                         log_to_file(f"      -> Membatasi ke {remaining_links} link untuk mencapai target {max_links}")
                
#                 log_to_file(f"      -> Ditemukan {len(page_urls)} link artikel.")
#                 urls_this_year.extend(page_urls)
#                 grand_total_urls.extend(page_urls)
                
#                 # Cek apakah sudah mencapai limit setelah menambah URL
#                 if max_links and len(grand_total_urls) >= max_links:
#                     log_to_file(f"      -> Target {max_links} link tercapai!")
#                     break

#             except requests.exceptions.RequestException as e:
#                 log_to_file(f"      -> ERROR: Gagal mengambil {page_url}: {e}")
#                 continue
        
#         log_to_file(f"--- Selesai Tahun {year}, ditemukan {len(urls_this_year)} link ---")

#     if grand_total_urls:
#         unique_urls = sorted(list(set(grand_total_urls)))
#         log_to_file(f"Menyimpan total {len(unique_urls)} link unik dari semua tahun ke file {URL_OUTPUT_FILE}...")
#         try:
#             with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
#                 for url in unique_urls:
#                     f.write(f"{PORTAL_NAME};{url}\n")
#             log_to_file("Penyimpanan link berhasil.")
#         except IOError as e:
#             log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
#     else:
#         log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
#     log_to_file(f"Total link yang didapat pada sesi ini: {len(grand_total_urls)}")
#     log_to_file(f"Total link unik yang disimpan: {len(unique_urls) if grand_total_urls else 0}")
#     log_to_file("===== Proses Crawling Selesai =====\n")

# if __name__ == "__main__":
#     # Panggil dengan parameter max_links
#     # Ubah nilai MAX_LINKS sesuai kebutuhan, atau set None untuk mengambil semua
#     crawl_kotakgame_multi_year(max_links=MAX_LINKS)

[2025-11-17 19:11:37] ===== Memulai Proses Crawling Kotakgame (Tahun 2025-2025) =====
[2025-11-17 19:11:37] Target: Mengambil maksimal 500 link
[2025-11-17 19:11:37] --- Memproses Tahun 2025 ---
[2025-11-17 19:11:37]   -> Mencari halaman terakhir dari: https://www.kotakgame.com/berita/index/2025/0/0/1/


C:\Users\ADVAN\AppData\Local\Temp\ipykernel_11964\395253600.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  last_page_link = soup.find('a', class_='prevnext', text='LAST »')


[2025-11-17 19:11:38]   -> Halaman terakhir ditemukan: 90
[2025-11-17 19:11:38]     (1/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/1/
[2025-11-17 19:11:41]       -> Ditemukan 20 link artikel.
[2025-11-17 19:11:41]     (2/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/2/
[2025-11-17 19:11:42]       -> Ditemukan 20 link artikel.
[2025-11-17 19:11:42]     (3/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/3/
[2025-11-17 19:11:44]       -> Ditemukan 20 link artikel.
[2025-11-17 19:11:44]     (4/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/4/
[2025-11-17 19:11:45]       -> Ditemukan 20 link artikel.
[2025-11-17 19:11:45]     (5/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/5/
[2025-11-17 19:11:46]       -> Ditemukan 20 link artikel.
[2025-11-17 19:11:46]     (6/90) Memproses halaman: https://www.kotakgame.com/berita/index/2025/0/0/6/
[2025-11-17 19:11:47]       -> Dit

### Indogamers

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# from datetime import datetime
# import os
# import time 
# import urllib3

# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# # --- Konfigurasi ---
# BASE_URL = "https://indogamers.com/"
# CATEGORIES = ['guides', 'pc', 'console', 'mobile'] 
# URL_OUTPUT_FILE = "../data/crawled_urls.txt"
# LOG_FILE = "../data/crawl_logs.txt"
# MAX_LINKS = 500 

# output_dir = os.path.dirname(URL_OUTPUT_FILE)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def log_to_file(message):
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, 'a', encoding='utf-8') as f:
#         f.write(log_entry)

# def crawl_indogamers_deep(max_links=None):
#     """
#     Crawl artikel dari Indogamers.com
    
#     Args:
#         max_links (int, optional): Jumlah maksimal link yang ingin diambil.
#                                    Jika None, akan mengambil semua link.
#     """
#     log_to_file("===== Memulai Proses Crawling Mendalam Indogamers.com =====")
#     if max_links:
#         log_to_file(f"Target: Mengambil maksimal {max_links} link")
    
#     total_urls_found_session = 0

#     for category in CATEGORIES:
#         # Cek apakah sudah mencapai limit
#         if max_links and total_urls_found_session >= max_links:
#             log_to_file(f"Sudah mencapai limit {max_links} link. Menghentikan crawling.")
#             break
            
#         log_to_file(f"Memulai kategori: '{category}'")
#         page_num = 1
#         urls_per_category = 0
        
#         while True:
#             # Cek limit sebelum memproses halaman baru
#             if max_links and total_urls_found_session >= max_links:
#                 log_to_file(f"  -> Sudah mencapai limit {max_links} link. Melewati kategori '{category}'.")
#                 break
                
#             page_url = f"{BASE_URL}{category}?page={page_num}"
#             log_to_file(f"  -> Memproses halaman: {page_url}")
            
#             try:
#                 response = requests.get(page_url, headers=HEADERS, timeout=20, verify=False)
                
#                 if response.status_code == 404:
#                     log_to_file(f"    -> Halaman tidak ditemukan (404). Akhir dari kategori '{category}'.")
#                     break

#                 response.raise_for_status()
#                 soup = BeautifulSoup(response.content, "html.parser")
                
#                 selector = "div[class*='article_recent__'] div[class*='article_recent_desc__'] h1 a"
#                 link_tags = soup.select(selector)
                
#                 if not link_tags:
#                     log_to_file(f"    -> Tidak ada artikel ditemukan. Akhir dari kategori '{category}'.")
#                     break
                
#                 page_urls = [tag['href'] for tag in link_tags if tag.has_attr('href')]
                
#                 # Batasi jumlah URL yang diambil jika sudah mendekati limit
#                 if max_links:
#                     remaining_links = max_links - total_urls_found_session
#                     if remaining_links < len(page_urls):
#                         page_urls = page_urls[:remaining_links]
#                         log_to_file(f"    -> Membatasi ke {remaining_links} link untuk mencapai target {max_links}")
                
#                 log_to_file(f"    -> Ditemukan {len(page_urls)} link artikel.")
                
#                 with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
#                     for url in page_urls:
#                         f.write(f"Indogamers;{url}\n")
                
#                 urls_per_category += len(page_urls)
#                 total_urls_found_session += len(page_urls)
                
#                 # Cek apakah sudah mencapai limit setelah menambah URL
#                 if max_links and total_urls_found_session >= max_links:
#                     log_to_file(f"    -> Target {max_links} link tercapai!")
#                     break
                
#                 page_num += 1 
#                 time.sleep(1) 
                
#             except requests.exceptions.RequestException as e:
#                 log_to_file(f"    -> ERROR: Gagal mengambil {page_url}: {e}. Mencoba lagi dalam 5 detik...")
#                 time.sleep(5)
#                 continue 
        
#         log_to_file(f"Selesai kategori '{category}'. Total link ditemukan: {urls_per_category}")

#     log_to_file(f"Total link yang didapat pada sesi ini: {total_urls_found_session}")
#     log_to_file("===== Proses Crawling Selesai =====\n")

# if __name__ == "__main__":
#     # Panggil dengan parameter max_links
#     # Ubah nilai MAX_LINKS sesuai kebutuhan, atau set None untuk mengambil semua
#     crawl_indogamers_deep(max_links=MAX_LINKS)

[2025-11-17 19:12:24] ===== Memulai Proses Crawling Mendalam Indogamers.com =====
[2025-11-17 19:12:24] Target: Mengambil maksimal 500 link
[2025-11-17 19:12:24] Memulai kategori: 'guides'
[2025-11-17 19:12:24]   -> Memproses halaman: https://indogamers.com/guides?page=1
[2025-11-17 19:12:25]     -> Ditemukan 20 link artikel.
[2025-11-17 19:12:26]   -> Memproses halaman: https://indogamers.com/guides?page=2
[2025-11-17 19:12:27]     -> Ditemukan 20 link artikel.
[2025-11-17 19:12:28]   -> Memproses halaman: https://indogamers.com/guides?page=3
[2025-11-17 19:12:30]     -> Ditemukan 20 link artikel.
[2025-11-17 19:12:31]   -> Memproses halaman: https://indogamers.com/guides?page=4
[2025-11-17 19:12:32]     -> Ditemukan 20 link artikel.
[2025-11-17 19:12:33]   -> Memproses halaman: https://indogamers.com/guides?page=5
[2025-11-17 19:12:34]     -> Ditemukan 20 link artikel.
[2025-11-17 19:12:35]   -> Memproses halaman: https://indogamers.com/guides?page=6
[2025-11-17 19:12:37]     -> Dite

### JagatPlay

In [ ]:
# import requests
# from bs4 import BeautifulSoup
# from datetime import datetime
# import os
# import re 
# import urllib3

# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# # --- Konfigurasi ---
# SITEMAP_INDEX_URL = "https://jagatplay.com/sitemap.html"
# URL_OUTPUT_FILE = "../data/crawled_urls.txt"
# LOG_FILE = "../data/crawl_logs.txt"
# PORTAL_NAME = "Jagatplay"
# MAX_LINKS = 500  # Jumlah maksimal link yang ingin diambil

# output_dir = os.path.dirname(URL_OUTPUT_FILE)
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def log_to_file(message):
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, 'a', encoding='utf-8') as f:
#         f.write(log_entry)

# def get_jagatplay_sitemap_number(url):
#     """Mengekstrak nomor dari URL sitemap untuk sorting yang benar."""
#     match = re.search(r'post-sitemap(\d+)\.html', url)
#     return int(match.group(1)) if match else 0

# def crawl_jagatplay(max_links=None):
#     """
#     Crawl artikel dari Jagatplay.com
    
#     Args:
#         max_links (int, optional): Jumlah maksimal link yang ingin diambil.
#                                    Jika None, akan mengambil semua link.
#     """
#     log_to_file(f"===== Memulai Proses Crawling {PORTAL_NAME} =====")
#     if max_links:
#         log_to_file(f"Target: Mengambil maksimal {max_links} link")
    
#     all_urls_found = []
    
#     try:
#         log_to_file(f"Mengambil sitemap index dari: {SITEMAP_INDEX_URL}")
#         response_main = requests.get(SITEMAP_INDEX_URL, headers=HEADERS, timeout=15, verify=False)
#         response_main.raise_for_status()
#         soup_main = BeautifulSoup(response_main.content, "html.parser")
        
#         sitemap_tags = soup_main.select('tr > td > a')
#         all_sitemap_links = [tag['href'] for tag in sitemap_tags if tag.has_attr('href')]
        
#         if not all_sitemap_links:
#             log_to_file("ERROR: Tidak ada link sitemap yang ditemukan di halaman index.")
#             return

#         log_to_file(f"Total sitemap ditemukan di index: {len(all_sitemap_links)}")
        
#         post_sitemap_links_numbered = [url for url in all_sitemap_links if re.search(r'post-sitemap\d+\.html', url)]
        
#         post_sitemap_links_numbered.sort(key=get_jagatplay_sitemap_number)
        
#         sitemap_limit = 11
#         sitemaps_to_process = [
#             url for url in post_sitemap_links_numbered 
#             if get_jagatplay_sitemap_number(url) <= sitemap_limit
#         ]
        
#         log_to_file(f"Menyaring sitemap... Akan memproses {len(sitemaps_to_process)} sitemap (hingga post-sitemap{sitemap_limit}).")

#         for i, sitemap_url in enumerate(sitemaps_to_process, 1):
#             # Cek apakah sudah mencapai limit
#             if max_links and len(all_urls_found) >= max_links:
#                 log_to_file(f"Sudah mencapai limit {max_links} link. Menghentikan crawling.")
#                 break
                
#             log_to_file(f"({i}/{len(sitemaps_to_process)}) Memproses sitemap: {sitemap_url}")
#             try:
#                 response_post = requests.get(sitemap_url, headers=HEADERS, timeout=15, verify=False)
#                 response_post.raise_for_status()

#                 # Gunakan lxml-xml dengan fallback ke html.parser
#                 try:
#                     soup_post = BeautifulSoup(response_post.content, "lxml-xml")
#                 except:
#                     soup_post = BeautifulSoup(response_post.content, "html.parser")
                
#                 article_links = []
#                 url_blocks = soup_post.find_all('url')
#                 for block in url_blocks:
#                     loc_tag = block.find('loc')
#                     if loc_tag:
#                         article_links.append(loc_tag.text)
                
#                 # Batasi jumlah URL yang diambil jika sudah mendekati limit
#                 if max_links:
#                     remaining_links = max_links - len(all_urls_found)
#                     if remaining_links < len(article_links):
#                         article_links = article_links[:remaining_links]
#                         log_to_file(f"  -> Membatasi ke {remaining_links} link untuk mencapai target {max_links}")
                
#                 log_to_file(f"  -> Ditemukan {len(article_links)} link artikel.")
#                 all_urls_found.extend(article_links)
                
#                 # Cek apakah sudah mencapai limit setelah menambah URL
#                 if max_links and len(all_urls_found) >= max_links:
#                     log_to_file(f"  -> Target {max_links} link tercapai!")
#                     break

#             except requests.exceptions.RequestException as e:
#                 log_to_file(f"  -> ERROR: Gagal mengambil atau memproses {sitemap_url}: {e}")
#                 continue

#     except requests.exceptions.RequestException as e:
#         log_to_file(f"FATAL ERROR: Gagal mengambil sitemap index utama. Proses dihentikan. Error: {e}")
#         return

#     if all_urls_found:
#         unique_urls = sorted(list(set(all_urls_found)))
#         log_to_file(f"Menyimpan total {len(unique_urls)} link unik ke file {URL_OUTPUT_FILE}...")
#         try:
#             with open(URL_OUTPUT_FILE, 'a', encoding='utf-8') as f:
#                 for url in unique_urls:
#                     f.write(f"{PORTAL_NAME};{url}\n")
#             log_to_file("Penyimpanan link berhasil.")
#         except IOError as e:
#             log_to_file(f"ERROR: Gagal menulis ke file {URL_OUTPUT_FILE}: {e}")
#     else:
#         log_to_file("Tidak ada link baru yang ditemukan pada sesi crawling ini.")
    
#     log_to_file(f"Total link yang didapat pada sesi ini: {len(all_urls_found)}")
#     log_to_file(f"Total link unik yang disimpan: {len(unique_urls) if all_urls_found else 0}")
#     log_to_file("===== Proses Crawling Selesai =====\n")


# if __name__ == "__main__":
#     # Panggil dengan parameter max_links
#     # Ubah nilai MAX_LINKS sesuai kebutuhan, atau set None untuk mengambil semua
#     crawl_jagatplay(max_links=MAX_LINKS)

[2025-11-17 19:13:30] ===== Memulai Proses Crawling Jagatplay =====
[2025-11-17 19:13:30] Target: Mengambil maksimal 500 link
[2025-11-17 19:13:30] Mengambil sitemap index dari: https://jagatplay.com/sitemap.html
[2025-11-17 19:13:34] Total sitemap ditemukan di index: 24
[2025-11-17 19:13:34] Menyaring sitemap... Akan memproses 10 sitemap (hingga post-sitemap11).
[2025-11-17 19:13:34] (1/10) Memproses sitemap: https://jagatplay.com/post-sitemap2.html
[2025-11-17 19:13:38]   -> Membatasi ke 500 link untuk mencapai target 500
[2025-11-17 19:13:38]   -> Ditemukan 500 link artikel.
[2025-11-17 19:13:38]   -> Target 500 link tercapai!
[2025-11-17 19:13:38] Menyimpan total 500 link unik ke file ../data/crawled_urls.txt...
[2025-11-17 19:13:38] Penyimpanan link berhasil.
[2025-11-17 19:13:38] Total link yang didapat pada sesi ini: 500
[2025-11-17 19:13:38] Total link unik yang disimpan: 500
[2025-11-17 19:13:38] ===== Proses Crawling Selesai =====


## **Scraping**

In [ ]:
# import requests
# from bs4 import BeautifulSoup, NavigableString, Comment
# from datetime import datetime
# import pandas as pd
# import os
# import time
# import re
# import urllib3

# # Nonaktifkan pesan peringatan SSL
# urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# # --- Konfigurasi ---
# CRAWLED_URL_FILE = "../data/crawled_urls.txt"
# OUTPUT_CSV_FILE = "../data/scraped_articles.csv"
# LOG_FILE = "../data/scrape_logs.txt"

# PORTALS_TO_SCRAPE = [
#     "Gamebrott",
#     "Kotakgame",
#     "Indogamers",
#     "Jagatplay"
# ]

# os.makedirs(os.path.dirname(OUTPUT_CSV_FILE), exist_ok=True)
# HEADERS = {
#     'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
# }

# def log_to_file(message):
#     timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
#     log_entry = f"[{timestamp}] {message}\n"
#     print(log_entry.strip())
#     with open(LOG_FILE, 'a', encoding='utf-8') as f:
#         f.write(log_entry)

# def read_all_urls_to_scrape(filepath, portal_names):
#     log_to_file(f"Membaca SEMUA URL dari {filepath}...")
#     all_tasks = []
#     try:
#         with open(filepath, 'r', encoding='utf-8') as f:
#             for line in f:
#                 if ';' in line:
#                     portal, url = line.strip().split(';', 1)
#                     if portal in portal_names:
#                         all_tasks.append((portal, url))
#         log_to_file(f"Total URL target ditemukan: {len(all_tasks)}")
#         return all_tasks
#     except FileNotFoundError:
#         log_to_file(f"ERROR: File {filepath} tidak ditemukan.")
#         return []

# def get_already_scraped_urls(filepath):
#     """Membaca CSV dan mengembalikan set URL yang sudah di-scrape."""
#     if not os.path.exists(filepath):
#         return set()
#     try:
#         df = pd.read_csv(filepath)
#         if 'url' in df.columns:
#             return set(df['url'])
#         else:
#             log_to_file("WARNING: Kolom 'url' tidak ditemukan di CSV. Tidak bisa melanjutkan. Harap hapus file CSV lama atau tambahkan kolom 'url'.")
#             return set()
#     except pd.errors.EmptyDataError:
#         return set()
#     except Exception as e:
#         log_to_file(f"Error saat membaca CSV yang ada: {e}. Mengasumsikan file kosong.")
#         return set()

# def extract_text_with_links(element):
#     """Ekstrak teks dari elemen termasuk tag <a>, lalu bersihkan HTML belakangan."""
#     if not element:
#         return 'N/A'
    
#     # Ambil semua paragraph
#     paragraphs = element.find_all('p')
#     content_parts = []
    
#     for p in paragraphs:
#         # Ambil teks dengan mempertahankan <a> tag sementara
#         text = p.get_text(separator=' ', strip=True)
#         if text:
#             content_parts.append(text)
    
#     return '\n'.join(content_parts) if content_parts else 'N/A'

# def scrape_kotakgame_article(url, soup):
#     """Scrape artikel dari Kotakgame"""
#     title = 'N/A'
#     thumbnail_url = 'N/A'
#     publish_date = 'N/A'
#     content = 'N/A'
    
#     # Judul - ada di class bagiankiri > h3.judulh3
#     bagian_kiri = soup.select_one('.bagiankiri')
#     if bagian_kiri:
#         title_tag = bagian_kiri.select_one('h3.judulh3')
#         if title_tag:
#             title = title_tag.get_text(strip=True)
    
#     # Thumbnail - ada di wrapimg > img src
#     thumb_tag = soup.select_one('.wrapimg img')
#     if thumb_tag and thumb_tag.has_attr('src'):
#         relative_url = thumb_tag['src']
#         thumbnail_url = f"https://www.kotakgame.com{relative_url}" if relative_url.startswith('/') else relative_url
    
#     # Tanggal - ada di boxwidget > boxcreate > span.txtcreate2
#     date_span = soup.select_one('.boxwidget .boxcreate .txtcreate2')
#     if date_span:
#         publish_date = date_span.get_text(strip=True)
    
#     # Konten - ada di isinewsp, ambil semua <p>
#     content_div = soup.select_one('.isinewsp')
#     if content_div:
#         content = extract_text_with_links(content_div)
    
#     return {
#         "judul": title,
#         "konten": content,
#         "tanggal_terbit": publish_date,
#         "url_thumbnail": thumbnail_url
#     }

# def scrape_indogamers_article(url, soup):
#     title_tag = soup.select_one('h1[class*="style_article__title__"]')
#     title = title_tag.get_text(strip=True) if title_tag else 'N/A'
#     thumb_tag = soup.select_one('div[class*="style_image__article__"] img')
#     if thumb_tag and thumb_tag.has_attr('srcset'):
#         last_url_part = thumb_tag['srcset'].split(',')[-1].strip()
#         relative_url = last_url_part.split(' ')[0]
#         thumbnail_url = f"https://indogamers.com{relative_url}"
#     else: thumbnail_url = 'N/A'
#     date_container = soup.select_one('div[class*="style_author__box__"]')
#     publish_date = 'N/A'
#     if date_container:
#         all_spans = date_container.find_all('span')
#         for span in all_spans:
#             span_text = span.get_text(strip=True)
#             if re.search(r'\b(Senin|Selasa|Rabu|Kamis|Jumat|Sabtu|Minggu)\b', span_text):
#                 publish_date = span_text.split(',')[0].strip()
#                 break
#     # content_div = soup.find('article[class*=style_content__article__]')
#     content_div = soup.find('article', class_=re.compile(r'style_content__article___'))
#     content = content_div.decode_contents() if content_div else 'N/A'
#     # content = 'N/A'
#     # if content_div:
#     #     paragraphs = content_div.find_all('p')
#     #     content = '\n'.join([p.get_text(strip=True) for p in paragraphs if not p.has_attr('class') or 'caption' not in ''.join(p['class'])])
#     return {"judul": title, "konten": content, "tanggal_terbit": publish_date, "url_thumbnail": thumbnail_url}

# def scrape_gamebrott_article(url, soup):
#     """Scrape artikel dari Gamebrott"""
#     title = 'N/A'
#     thumbnail_url = 'N/A'
#     publish_date = 'N/A'
#     content = 'N/A'
    
#     # Judul - class post-wrapper > h1.jeg_post_title
#     post_wrapper = soup.select_one('.post-wrapper')
#     if post_wrapper:
#         title_tag = post_wrapper.select_one('h1.jeg_post_title')
#         if title_tag:
#             title = title_tag.get_text(strip=True)
    
#     # Thumbnail - class jeg_featured.featured_image > thumbnail-container > img src
#     thumb_tag = soup.select_one('.jeg_featured.featured_image .thumbnail-container img')
#     if thumb_tag and thumb_tag.has_attr('src'):
#         thumbnail_url = thumb_tag['src']
    
#     # Tanggal - div.jeg_meta_container > div.jeg_meta_date > a
#     date_tag = soup.select_one('.jeg_meta_container .jeg_meta_date a')
#     if date_tag:
#         publish_date = date_tag.get_text(strip=True)
    
#     # Konten - div.entry-content.no-share > div.content-inner.jeg_link_underline > p
#     content_div = soup.select_one('.entry-content.no-share .content-inner.jeg_link_underline')
#     if content_div:
#         content = extract_text_with_links(content_div)
    
#     return {
#         "judul": title,
#         "konten": content,
#         "tanggal_terbit": publish_date,
#         "url_thumbnail": thumbnail_url
#     }

# def scrape_jagatplay_article(url, soup):
#     """Scrape artikel dari Jagatplay"""
#     title = 'N/A'
#     thumbnail_url = 'N/A'
#     publish_date = 'N/A'
#     content = 'N/A'
    
#     # Cari div#mainContent dulu
#     main_content = soup.select_one('div#mainContent')
#     if not main_content:
#         main_content = soup  # Fallback ke soup utama
    
#     # Judul - div.jgpost__header > h1
#     header = main_content.select_one('.jgpost__header')
#     if header:
#         title_tag = header.select_one('h1')
#         if title_tag:
#             title = title_tag.get_text(strip=True)
    
#     # Thumbnail - class jgpost__feat-img, ambil background url dari style
#     feat_img = main_content.select_one('.jgpost__feat-img')
#     if feat_img and feat_img.has_attr('style'):
#         style = feat_img['style']
#         match = re.search(r"url\(['\"]?(.*?)['\"]?\)", style)
#         if match:
#             thumbnail_url = match.group(1)
    
#     # Tanggal - div.jgpost__content > div.jgauthor.breakout > div.jgauthor__posted > div
#     author_posted = main_content.select_one('.jgpost__content .jgauthor.breakout .jgauthor__posted')
#     if author_posted:
#         # Cari div yang berisi tanggal (biasanya div terakhir atau yang tidak punya tag <a>)
#         divs = author_posted.find_all('div', recursive=False)
#         for div in divs:
#             if not div.find('a'):  # Div tanpa link biasanya berisi tanggal
#                 publish_date = div.get_text(strip=True)
#                 break
    
#     # Konten - div.jgpost__content > p
#     content_div = main_content.select_one('.jgpost__content')
#     if content_div:
#         content = extract_text_with_links(content_div)
    
#     return {
#         "judul": title,
#         "konten": content,
#         "tanggal_terbit": publish_date,
#         "url_thumbnail": thumbnail_url
#     }

# def main():
#     log_to_file("===== Memulai Proses Scraping Skala Penuh (Mode Resume + Real-time Save) =====")
    
#     all_tasks = read_all_urls_to_scrape(CRAWLED_URL_FILE, PORTALS_TO_SCRAPE)
    
#     header = ['id_dokumen', 'sumber', 'url', 'judul', 'konten', 'tanggal_terbit', 'url_thumbnail']
    
#     already_scraped_urls = get_already_scraped_urls(OUTPUT_CSV_FILE)
#     if already_scraped_urls:
#         log_to_file(f"Ditemukan {len(already_scraped_urls)} URL yang sudah diproses. Akan melanjutkan.")
    
#     doc_id_counter = len(already_scraped_urls) + 1
    
#     if not os.path.exists(OUTPUT_CSV_FILE) or not already_scraped_urls:
#         log_to_file(f"File {OUTPUT_CSV_FILE} tidak ada atau kosong. Membuat file baru dengan header.")
#         pd.DataFrame(columns=header).to_csv(OUTPUT_CSV_FILE, index=False)
#         doc_id_counter = 1
    
#     total_urls_to_process = len(all_tasks)
#     newly_scraped_count = 0
    
#     for i, (portal, url) in enumerate(all_tasks):
#         # Lewati URL yang sudah ada
#         if url in already_scraped_urls:
#             continue
            
#         log_to_file(f"  ({i+1}/{total_urls_to_process}) Scraping: {url}")
        
#         try:
#             response = requests.get(url, headers=HEADERS, timeout=15, verify=False)
#             if response.status_code != 200:
#                 log_to_file(f"    -> Gagal mengakses (Status: {response.status_code})")
#                 continue
            
#             soup = BeautifulSoup(response.content, 'html.parser')

#             data = None
#             if portal == "Gamebrott":
#                 data = scrape_gamebrott_article(url, soup)
#             elif portal == "Kotakgame":
#                 data = scrape_kotakgame_article(url, soup)
#             elif portal == "Indogamers":
#                 data = scrape_indogamers_article(url, soup)
#             elif portal == "Jagatplay":
#                 data = scrape_jagatplay_article(url, soup)
            
#             if data:
#                 data['id_dokumen'] = f"doc_{doc_id_counter:05d}"
#                 data['sumber'] = portal
#                 data['url'] = url
                
#                 df_row = pd.DataFrame([data])
#                 df_row = df_row[header]
                
#                 df_row.to_csv(OUTPUT_CSV_FILE, mode='a', index=False, header=False)
                
#                 log_to_file(f"    -> Berhasil: {data['judul'][:50]}...")
                
#                 doc_id_counter += 1
#                 newly_scraped_count += 1

#             time.sleep(0.5)

#         except Exception as e:
#             log_to_file(f"    -> Terjadi error kritis saat scraping {url}: {e}")

#     log_to_file(f"Scraping selesai. Total {newly_scraped_count} artikel BARU berhasil disimpan ke {OUTPUT_CSV_FILE}.")
#     log_to_file(f"Total artikel keseluruhan: {doc_id_counter - 1}")
#     log_to_file(f"Proses selesai.\n")

# if __name__ == "__main__":
#     main()

[2025-11-19 19:24:49] ===== Memulai Proses Scraping Skala Penuh (Mode Resume + Real-time Save) =====
[2025-11-19 19:24:49] Membaca SEMUA URL dari ../data/crawled_urls.txt...
[2025-11-19 19:24:49] Total URL target ditemukan: 2101
[2025-11-19 19:24:49] Ditemukan 440 URL yang sudah diproses. Akan melanjutkan.
[2025-11-19 19:24:49]   (639/2101) Scraping: https://www.kotakgame.com/berita/detail/105285/Tidak-Sengaja-Bandai-Namco-Akan-Umumkan-Tales-of-Xillia-Remaster-Waktu-Dekat-Ini
[2025-11-19 19:24:54]     -> Berhasil: Tidak Sengaja, Bandai Namco Akan Umumkan Tales of ...
[2025-11-19 19:24:55]   (640/2101) Scraping: https://www.kotakgame.com/berita/detail/105286/Bak-Drama-Sosmed-Di-Indonesia-Elon-Musk-dan-Sam-Altman-RIbut-di-Twitter-Cuman-Karena-Peringkat-App
[2025-11-19 19:24:59]     -> Berhasil: Bak Drama Sosmed Di Indonesia, Elon Musk dan Sam A...
[2025-11-19 19:25:00]   (641/2101) Scraping: https://www.kotakgame.com/berita/detail/105287/Karyawan-BUMN-Siap-Siap-Kesusahan-Posisi-Kosong-Di

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[2025-11-19 19:52:09]     -> Berhasil: N/A...
[2025-11-19 19:52:10]   (942/2101) Scraping: https://www.kotakgame.com/berita/detail/105600/WhatsApp-Bakal-Bisa-Pakai-Username-Tak-Perlu-Sebar-Nomor-HP-Lagi


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[2025-11-19 19:52:10]     -> Berhasil: N/A...
[2025-11-19 19:52:11]   (943/2101) Scraping: https://www.kotakgame.com/berita/detail/105601/Pre-Order-iPhone-17-Series-Dibuka-Stoknya-Banyak-yang-Sudah-Habis
[2025-11-19 19:52:11]     -> Berhasil: Pre-Order iPhone 17 Series Dibuka, Stoknya Banyak ...
[2025-11-19 19:52:12]   (944/2101) Scraping: https://www.kotakgame.com/berita/detail/105602/Football-Manager-26-Hadirkan-Sepak-Bola-Wanita-UI-Baru-dan-Sistem-Taktik-yang-Lebih-Dalam


...

[2025-11-19 19:52:15]     -> Berhasil: N/A...
[2025-11-19 19:52:15]   (948/2101) Scraping: https://www.kotakgame.com/berita/detail/105606/realme-15-Series-5G-AI-Night-Out-Phone-Terbaru-Mengabadikan-Momen-Malam-Anak-Muda-dengan-AI-Kamera-Cerdas


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[2025-11-19 19:52:16]     -> Berhasil: N/A...
[2025-11-19 19:52:16]   (949/2101) Scraping: https://www.kotakgame.com/berita/detail/105607/Trailer-Kisah-CODE-VEIN-II-Telah-Terungkap


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


[2025-11-19 19:52:17]     -> Berhasil: N/A...
[2025-11-19 19:52:17]   (950/2101) Scraping: https://www.kotakgame.com/berita/detail/105608/Tekken-8-Umumkan-Miary-Zo-di-EVO-France
[2025-11-19 19:52:23]     -> Berhasil: Tekken 8 Umumkan Miary Zo di EVO France...
[2025-11-19 19:52:23]   (951/2101) Scraping: https://www.kotakgame.com/berita/detail/105609/Preview-Sword-of-Justice-Game-RPG-yang-Bikin-Ngerasa-Jadi-MC-Anime-JUSTICE-DELIVERED
[2025-11-19 19:52:24]     -> Berhasil: Preview Sword of Justice - Game RPG yang Bikin Nge...
[2025-11-19 19:52:24]   (952/2101) Scraping: https://www.kotakgame.com/berita/detail/105610/Bocoran-Rilis-Apple-Oktober-Ini-Dari-Mac-hingga-iPad-Semua-Serba-M5
[2025-11-19 19:52:28]     -> Berhasil: Bocoran Rilis Apple Oktober Ini: Dari Mac hingga i...
[2025-11-19 19:52:29]   (953/2101) Scraping: https://www.kotakgame.com/berita/detail/105611/Roadmap-Game-Pokemon-dan-GameFreak-Bocor-Ada-Mainline-Games-di-Indonesia
[2025-11-19 19:52:29]     -> Berhasil: Roadmap Game 

## **Preprocessing**

### EDA

In [8]:
import pandas as pd

df_article = pd.read_csv('../data/scraped_articles.csv')
df_article.head()

,id_dokumen,sumber,url,judul,konten,tanggal_terbit,url_thumbnail
0,doc_00001,Gamebrott,https://gamebrott.com/blog/,NaN,NaN,NaN,NaN
1,doc_00002,Gamebrott,https://gamebrott.com/microsoft-investigasi-penyebab-masalah-ssd/,"Microsoft Investigasi Penyebab Masalah SSD, Benarkah Update Penyebabnya?","Beberapa hari lalu, permasalahan SSD rusak menjadi perbincangan hangat di komunitas. Untungnya, ...",21 Agustus 2025,https://gamebrott.com/wp-content/uploads/2025/08/Microsoft-Investigasi-Penyebab-Masalah-pada-SSD...
2,doc_00003,Gamebrott,https://gamebrott.com/amd-klarifikasi-soal-socket-am5-terbakar/,"AMD Klarifikasi Masalah Socket AM5 Terbakar, Sudah Ada Solusi?",AMD akhirnya buka suara mengenai socket prosesor seri Ryzen 9000 yang mengalami masalah terbakar...,21 Agustus 2025,https://gamebrott.com/wp-content/uploads/2025/08/socket-am5-terbakar-1-750x375.webp
3,doc_00004,Gamebrott,https://gamebrott.com/detail-trailer-fallout-season-2-harus-kalian-ketahui/,Gamescom 2025 — Detail Trailer Fallout Season 2 yang Harus Kalian Ketahui,Detail Trailer Fallout Season 2 – Trailer TV Series Fallout Season 2 telah diumumkan pada acara ...,20 Agustus 2025,https://gamebrott.com/wp-content/uploads/2025/08/Trailer-Fallout-Season-2-1-750x375.webp
4,doc_00005,Gamebrott,https://gamebrott.com/kisah-clair-obscur-expedition-33/,"Kisah Clair Obscur Expedition 33, Mulai dari Awal hingga Akhir","Kisah Clair Obscur: Expedition 33 — Game perdana dari Sandfall Interactive, Clair Obscur Expedi...",20 Agustus 2025,https://gamebrott.com/wp-content/uploads/2025/08/Kisah-Clair-Obscur-Expedition-33-thumbnail-1-75...


In [9]:
print("Statistik Deskriptif Untuk Dataframe Articles")
print(df_article.describe())

print("Tipe Data Dataframe Articles")
print(df_article.dtypes)


Statistik Deskriptif Untuk Dataframe Articles
       id_dokumen     sumber                                                                     url                                            judul                                                                                               konten  tanggal_terbit                                                                                        url_thumbnail
count        2029       2029                                                                    2029                                             2020                                                                                                 2020            2020                                                                                                 2020
unique       1831          4                                                                    1831                                             1822                                                                     

### Missing Values

In [10]:
df_article.isnull().sum()

id_dokumen        0
sumber            0
url               0
judul             9
konten            9
tanggal_terbit    9
url_thumbnail     9
dtype: int64

In [11]:
df_null = df_article.dropna()
print(f"Jumlah baris sebelum dihandling: {df_article.shape[0]}")
print(f"Jumlah baris setelah dihandling: {df_null.shape[0]}")

df_null.isnull().sum()
df_article = df_null


Jumlah baris sebelum dihandling: 2029
Jumlah baris setelah dihandling: 2020


### Duplicate

In [12]:
df_article.duplicated().sum()

np.int64(0)

### Invalid Content

In [13]:
import pandas as pd
import re

MIN_WORD_COUNT = 15
print("--- Data SEBELUM Pembersihan ---")
print(f"Jumlah baris awal: {len(df_article)}")
# display(df_article)
print("-" * 50)

# --- Langkah 2: Hapus baris dengan konten terlalu pendek ---
print(f"\n--- Menghapus baris dengan konten kurang dari {MIN_WORD_COUNT} kata ---")

# Hitung jumlah kata di setiap baris pada kolom 'konten'
# .str.split().str.len() adalah cara cepat untuk menghitung kata
word_counts = df_article['konten'].str.split().str.len().fillna(0).astype(int)

# Identifikasi baris yang terlalu pendek
rows_too_short = word_counts < MIN_WORD_COUNT

# Simpan baris yang akan dihapus untuk direview (opsional)
short_content_rows = df_article[rows_too_short]
if not short_content_rows.empty:
    print(f"Ditemukan {len(short_content_rows)} baris dengan konten terlalu pendek:")
    # Tambahkan kolom 'word_count' untuk konteks
    short_content_rows = short_content_rows.assign(word_count=word_counts[rows_too_short])
    display(short_content_rows[['id_dokumen', 'sumber', 'konten', 'word_count']])
else:
    print("Tidak ditemukan baris dengan konten terlalu pendek.")
print("-" * 50)

# Filter DataFrame untuk menyimpan HANYA baris yang panjangnya CUKUP
df_article_cleaned = df_article[~rows_too_short]


# --- Hasil Akhir ---
print("\n--- Data SETELAH Semua Pembersihan ---")
print(f"Jumlah baris awal: {len(df_article)}")
print(f"Jumlah baris setelah dibersihkan total: {len(df_article_cleaned)}")
print(f"Total baris yang dihapus: {len(df_article) - len(df_article_cleaned)}")
# display(df_article_cleaned)
df_article = df_article_cleaned

--- Data SEBELUM Pembersihan ---
Jumlah baris awal: 2020
--------------------------------------------------

--- Menghapus baris dengan konten kurang dari 15 kata ---
Tidak ditemukan baris dengan konten terlalu pendek.
--------------------------------------------------

--- Data SETELAH Semua Pembersihan ---
Jumlah baris awal: 2020
Jumlah baris setelah dibersihkan total: 2020
Total baris yang dihapus: 0


### Date Standardize

In [14]:
unique_sources = df_article['sumber'].unique()
sample_rows = []

for source in unique_sources:
    sample = df_article[df_article['sumber'] == source].head(1)
    sample_rows.append(sample)

if sample_rows:
    temp_df = pd.concat(sample_rows)
    pd.set_option('display.max_colwidth', 100)
    pd.set_option('display.width', 1000)
    print(temp_df[['sumber', 'tanggal_terbit']])

          sumber                        tanggal_terbit
1      Gamebrott                       21 Agustus 2025
601    Kotakgame  Rahmat Handiko|   Kamis, 07 Aug 2025
1081  Indogamers                Kamis 13 November 2025
1561   Jagatplay                         July 31, 2012


In [15]:
import pandas as pd
import re

df_article 

# Dictionary bulan Inggris ke Indonesia
bulan_dict = {
    'January': 'Januari', 'February': 'Februari', 'March': 'Maret', 'April': 'April',
    'May': 'Mei', 'June': 'Juni', 'July': 'Juli', 'August': 'Agustus', 'September': 'September',
    'October': 'Oktober', 'November': 'November', 'December':'Desember',
    'Jan': 'Januari', 'Feb': 'Februari', 'Mar': 'Maret', 'Apr': 'April', 'May': 'Mei',
    'Jun': 'Juni', 'Jul': 'Juli', 'Aug': 'Agustus', 'Sep': 'September', 'Oct': 'Oktober',
    'Nov': 'November', 'Dec':'Desember','Agu':'Agustus','Okt':'Oktober','Des':'Desember'
}

# Pembersihan dan konversi
def convert_date(text):
    if not isinstance(text, str):
        return 'N/A'
    
    original_text = text.strip()

    if '|' in original_text:
        text = original_text.split('|', 1)[-1].strip()
    
    if re.search(r'(?i)\b\d+\s+(Hari|Jam)\s+yang\s+lalu\b', text):
        return '20 November 2025'
    
    # Buang hari & jam jika ada
    text = re.sub(r'(?i)\b(Minggu|Senin|Selasa|Rabu|Kamis|Jumat|Sabtu),?\s*', '', text)
    text = re.sub(r'\b(Minggu|Senin|Selasa|Rabu|Kamis|Jumat|Sabtu)\s*', '', text)
    text = re.sub(r'\d{2}:\d{2}$', '', text)
    
    # Ubah 'July 31, 2012' ke '31 July 2012'
    m = re.match(r'([A-Za-z]+) (\d{1,2}), (\d{4})', text.strip())
    if m:
        text = f"{m.group(2)} {m.group(1)} {m.group(3)}"
    
    # Ubah ke list lalu ganti bulan
    parts = text.strip().split()
    if len(parts) >= 3:
        hari, bulan_raw, tahun = parts[0], parts[1], parts[2]
    else:
        return text.strip()
    # Ganti bulan
    bulan = bulan_dict.get(bulan_raw, bulan_raw)
    # Format ulang
    return f"{hari} {bulan} {tahun}"

df_article['tanggal_terbit_normalized'] = df_article['tanggal_terbit'].apply(convert_date)

unique_sources = df_article['sumber'].unique()
sample_rows = []

for source in unique_sources:
    sample = df_article[df_article['sumber'] == source].head(1)
    sample_rows.append(sample)

if sample_rows:
    temp_df = pd.concat(sample_rows)
    pd.set_option('display.max_colwidth', 100)
    pd.set_option('display.width', 1000)
    print(temp_df[['sumber', 'tanggal_terbit', 'tanggal_terbit_normalized']])

          sumber                        tanggal_terbit tanggal_terbit_normalized
1      Gamebrott                       21 Agustus 2025           21 Agustus 2025
601    Kotakgame  Rahmat Handiko|   Kamis, 07 Aug 2025           07 Agustus 2025
1081  Indogamers                Kamis 13 November 2025          13 November 2025
1561   Jagatplay                         July 31, 2012              31 Juli 2012


In [16]:
import pandas as pd
df_article_cleaned = df_article.drop('tanggal_terbit', axis=1)
df_article_cleaned = df_article_cleaned.rename(columns={'tanggal_terbit_normalized': 'tanggal_terbit'})

# Kamus untuk mengubah tanggal_terbit dari format object menjadi datetime
ID_TO_EN_MAP = {
    'Januari': 'January',
    'Februari': 'February',
    'Maret': 'March',
    'April': 'April',
    'Mei': 'May',
    'Juni': 'June',
    'Juli': 'July',
    'Agustus': 'August',
    'September': 'September',
    'Oktober': 'October',
    'November': 'November',
    'Desember': 'December'
}

def convert_indo_date_to_datetime(date_str):
    """Mengubah string tanggal Indo (15 Agustus 2025) ke datetime object."""
    if not isinstance(date_str, str) or date_str == 'N/A':
        return pd.NaT
    
    # Terjemahkan nama bulan di dalam string ke Inggris
    # Contoh: "15 Agustus 2025" -> "15 August 2025"
    date_str_en = date_str
    for id_month, en_month in ID_TO_EN_MAP.items():
        if id_month in date_str:
            date_str_en = date_str.replace(id_month, en_month)
            break
            
    # Sekarang Pandas bisa membacanya dengan mudah
    try:
        return pd.to_datetime(date_str_en, format='%d %B %Y')
    except Exception:
        return pd.NaT

# 2. Terapkan ke kolom baru 'timestamp'
print("Sedang membuat kolom timestamp...")
df_article_cleaned['timestamp'] = df_article_cleaned['tanggal_terbit'].apply(convert_indo_date_to_datetime)

# 3. Cek Hasilnya
print("\nCek tipe data:")
print(df_article_cleaned.dtypes)

print("\nContoh data:")
# Tampilkan kolom tanggal (string indo) dan timestamp (datetime) berdampingan
display(df_article_cleaned[['tanggal_terbit', 'timestamp']].head())

# Opsional: Cek apakah masih ada NaT (selain yang memang N/A)
jumlah_nat = df_article_cleaned['timestamp'].isna().sum()
print(f"\nJumlah baris yang gagal dikonversi (NaT): {jumlah_nat}")
timestamp_nat = df_article_cleaned['timestamp'].isna()
failed_rows = df_article_cleaned[timestamp_nat]

if not failed_rows.empty:
    print(f"\nDitemukan {len(failed_rows)} baris yang gagal dikonversi menjadi datetime (NaT).")
    print("Berikut adalah sampel dari baris-baris yang gagal tersebut:")
    
    # 'tanggal_terbit' adalah kolom yang paling penting untuk dianalisis
    display(failed_rows[['sumber', 'tanggal_terbit', 'timestamp']])
    
    print("\n--- Analisis Frekuensi Format Tanggal yang Gagal ---")
    print("Berikut adalah format-format tanggal unik yang paling sering menyebabkan kegagalan:")
    
    display(failed_rows['tanggal_terbit'].value_counts().head(20))
    
else:
    print("\nSelamat! Tidak ada baris dengan nilai NaT di kolom 'timestamp'.")

print(df_article_cleaned.head(5))

Sedang membuat kolom timestamp...

Cek tipe data:
id_dokumen                object
sumber                    object
url                       object
judul                     object
konten                    object
url_thumbnail             object
tanggal_terbit            object
timestamp         datetime64[ns]
dtype: object

Contoh data:


,tanggal_terbit,timestamp
1,21 Agustus 2025,2025-08-21
2,21 Agustus 2025,2025-08-21
3,20 Agustus 2025,2025-08-20
4,20 Agustus 2025,2025-08-20
5,20 Agustus 2025,2025-08-20



Jumlah baris yang gagal dikonversi (NaT): 0

Selamat! Tidak ada baris dengan nilai NaT di kolom 'timestamp'.
  id_dokumen     sumber                                                                          url                                                                            judul                                                                                               konten                                                                                        url_thumbnail   tanggal_terbit  timestamp
1  doc_00002  Gamebrott            https://gamebrott.com/microsoft-investigasi-penyebab-masalah-ssd/         Microsoft Investigasi Penyebab Masalah SSD, Benarkah Update Penyebabnya?  Beberapa hari lalu, permasalahan SSD rusak menjadi perbincangan hangat di komunitas. Untungnya, ...  https://gamebrott.com/wp-content/uploads/2025/08/Microsoft-Investigasi-Penyebab-Masalah-pada-SSD...  21 Agustus 2025 2025-08-21
2  doc_00003  Gamebrott              https://gamebrott.com/amd-kla

In [17]:
import pandas as pd
from datetime import datetime, timedelta
import re

TODAY = datetime(2025, 11, 10)

ID_TO_EN_MAP = {
    'Januari': 'January', 'Februari': 'February', 'Maret': 'March', 'April': 'April',
    'Mei': 'May', 'Juni': 'June', 'Juli': 'July', 'Agustus': 'August',
    'September': 'September', 'Oktober': 'October', 'November': 'November', 'Desember': 'December'
}

def convert_to_datetime(date_str):
    if not isinstance(date_str, str) or date_str.lower() == 'n/a':
        return pd.NaT
    
    match = re.search(r'(\d+)\s+Hari yang', date_str, re.IGNORECASE)
    if match:
        days_ago = int(match.group(1))
        calculated_date = TODAY - timedelta(days=days_ago)
        return calculated_date
    
    date_str_en = date_str
    for id_month, en_month in ID_TO_EN_MAP.items():
        if id_month in date_str:
            date_str_en = date_str.replace(id_month, en_month)
            break

    try:
        return pd.to_datetime(date_str_en, format='%d %B %Y')
    except Exception:
        # Jika gagal, kembalikan NaT
        print(f"Gagal mem-parsing tanggal absolut: '{date_str}'")
        return pd.NaT

print("--- DataFrame SEBELUM konversi tanggal relatif ---")
df_article_cleaned.head()
print("-" * 50)

# Terapkan fungsi konversi baru untuk membuat kolom 'timestamp'
print("Mengonversi semua format tanggal ke tipe datetime...")
df_article_cleaned['timestamp'] = df_article_cleaned['tanggal_terbit'].apply(convert_to_datetime)

# (Opsional tapi direkomendasikan) Perbarui juga kolom string 'tanggal_terbit' agar konsisten
# Fungsi ini akan mengubah objek datetime kembali menjadi format string Indonesia yang Anda inginkan
def format_to_indonesian_str(dt_object):
    if pd.isna(dt_object):
        return 'N/A'
    english_date_str = dt_object.strftime('%d %B %Y')
    for en_month, id_month in ID_TO_EN_MAP.items():
        english_date_str = english_date_str.replace(en_month, id_month)
    return english_date_str

# Timpa kolom 'tanggal_terbit' yang lama dengan format yang sudah konsisten
df_article_cleaned['tanggal_terbit'] = df_article_cleaned['timestamp'].apply(format_to_indonesian_str)

print("Konversi selesai.")
print("\n--- DataFrame SETELAH konversi ---")
df_article_cleaned.head()
print("\nTipe data akhir:")
df_article_cleaned.info()

--- DataFrame SEBELUM konversi tanggal relatif ---
--------------------------------------------------
Mengonversi semua format tanggal ke tipe datetime...
Konversi selesai.

--- DataFrame SETELAH konversi ---

Tipe data akhir:
<class 'pandas.core.frame.DataFrame'>
Index: 2020 entries, 1 to 2028
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id_dokumen      2020 non-null   object        
 1   sumber          2020 non-null   object        
 2   url             2020 non-null   object        
 3   judul           2020 non-null   object        
 4   konten          2020 non-null   object        
 5   url_thumbnail   2020 non-null   object        
 6   tanggal_terbit  2020 non-null   object        
 7   timestamp       2020 non-null   datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 142.0+ KB


In [18]:
df_article = df_article_cleaned
df_article.isna().sum()

id_dokumen        0
sumber            0
url               0
judul             0
konten            0
url_thumbnail     0
tanggal_terbit    0
timestamp         0
dtype: int64

### Content Preprocessing

#### Clean HTML Tag

#### Create Whitelist

In [ ]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

def clean_for_ngram(text):
    if isinstance(text, str):
        # Case Folding
        text = text.lower()
        # Hapus tanda baca/karakter khusus, tetapi pertahankan spasi
        text = re.sub(r'[^a-z0-9\s]', ' ', text)
        return text
    return ""

df_article['judul_clean'] = df_article['judul'].apply(clean_for_ngram)

STOPWORDS_COMBINED = set(stopwords.words('indonesian')) | set(stopwords.words('english'))

def get_top_ngrams(corpus, n=2, top_k=500):
    """Fungsi untuk mendapatkan N-gram yang paling sering muncul."""
    
    # Inisialisasi CountVectorizer
    vectorizer = CountVectorizer(ngram_range=(n, n), stop_words=list(STOPWORDS_COMBINED))
    
    # Fit dan transform corpus
    X = vectorizer.fit_transform(corpus)
    
    # Jumlahkan frekuensi setiap n-gram
    word_counts = X.sum(axis=0)
    word_freq = [(word, word_counts[0, idx]) 
                 for word, idx in vectorizer.vocabulary_.items()]
    
    # Urutkan berdasarkan frekuensi dan ambil yang teratas
    word_freq.sort(key=lambda x: x[1], reverse=True)
    return word_freq[:top_k]

# Ekstrak 500 Bigram (2-kata) teratas
top_bigrams = get_top_ngrams(df_article['judul_clean'], n=2, top_k=500)

# Ekstrak 500 Trigram (3-kata) teratas
top_trigrams = get_top_ngrams(df_article['judul_clean'], n=3, top_k=500)

print("### Top 20 Bigram ###")
for phrase, count in top_bigrams[:20]:
    print(f"'{phrase}' (Count: {count})")

print("\n### Top 20 Trigram ###")
for phrase, count in top_trigrams[:20]:
    print(f"'{phrase}' (Count: {count})")

# 1. Inisialisasi set untuk menyimpan kata-kata unik
potential_whitelist_words = set()

# 2. Gabungkan dan Tokenisasi Bigram
for phrase, count in top_bigrams:
    # Pisahkan frasa menjadi kata-kata (token)
    for word in phrase.split():
        potential_whitelist_words.add(word)

# 3. Gabungkan dan Tokenisasi Trigram
for phrase, count in top_trigrams:
    for word in phrase.split():
        potential_whitelist_words.add(word)
        
# 4. Tambahkan Jargon/Akronim Hardcoded (sebelum filtering)
potential_whitelist_words.update([
    "the", "of", "and", "or", "in", "with", 
    "buff", "nerf", "meta", "patch", "dlc", "rts", "fps", "moba", "rpg",
    "pc", "ps5", "xbox", "nintendo", "steam", "mobile", 
])

print(f"Total kata unik (token) sebelum filtering: {len(potential_whitelist_words)}")

sorted_words = sorted(list(potential_whitelist_words))

# Simulasi proses filtering manual
final_whitelist_words = sorted_words.copy() 

# Contoh kata yang mungkin perlu dihapus dari list
words_to_remove = [
    "cara", "tips", "trik", "terbaru", "akan", "bikin", "terbaik", "terlengkap", "wajib", "dibuka", "dimainkan", "gratis", "ketagihan", "seru",
      "lupa", "coba", "dicoba", "keren", "mainkan", "banget", "lengkap"]

# Hapus kata-kata yang dianggap tidak relevan setelah kurasi
final_whitelist_words = [word for word in final_whitelist_words if word not in words_to_remove]

# Nama file yang akan digunakan untuk menyimpan whitelist
FILE_PATH = '../data/whitelist.txt'

with open(FILE_PATH, 'w', encoding='utf-8') as f:
    for word in final_whitelist_words:
        f.write(word + '\n')

print(f"\n✅ Whitelist (total {len(final_whitelist_words)} kata unik) berhasil disimpan ke '{FILE_PATH}'.")

#### Create Stopwords List

In [ ]:
import pandas as pd
from collections import Counter
import re
import os 
import nltk
from nltk.corpus import stopwords 

# =======================================================
# 1. FUNGSI DAN MUAT KOMPONEN
# =======================================================

# 🚨 Penambahan try-except untuk download NLTK stopwords
try:
    stopwords.words('indonesian')
except LookupError:
    print("⏳ NLTK stopwords Bahasa Indonesia belum diunduh. Sedang mengunduh...")
    nltk.download('stopwords')
    print("✅ Pengunduhan NLTK stopwords selesai.")

def load_words_set(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        print(f"⚠️ Peringatan: File {file_path} tidak ditemukan. Menggunakan set kosong.")
        return set()

# Asumsi lokasi file Whitelist
WHITELIST_FILE_PATH = '../data/whitelist.txt'
WHITELIST = load_words_set(WHITELIST_FILE_PATH)

# Menggunakan Stoplist Bahasa Indonesia dari NLTK
NLTK_ID_STOPWORDS = set(stopwords.words('indonesian'))
STANDARD_STOPWORDS = NLTK_ID_STOPWORDS 

def clean_and_tokenize(text):
    if not isinstance(text, str):
        return []
    
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', ' ', text, flags=re.MULTILINE)
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\d+', ' ', text)
    
    return text.split()

# =======================================================
# 2. EKSTRAKSI KATA FREKUENSI TINGGI DARI KONTEN
# =======================================================

print("--- Proses Ekstraksi Stopword Tambahan Dimulai ---")

all_tokens = []
if 'konten' in df_article.columns:
    for content in df_article['konten']: 
        all_tokens.extend(clean_and_tokenize(content))
else:
    print("❌ ERROR: Kolom 'konten' tidak ditemukan di DataFrame.")
    
token_counts = Counter(all_tokens)
print(f"Total token unik yang dianalisis: {len(token_counts)}")

FREQUENCY_THRESHOLD = 25000 
candidate_additional_stopwords = []
sorted_token_counts = sorted(token_counts.items(), key=lambda item: item[1], reverse=True)

# Loop untuk mencari kandidat stopwords tambahan
for word, count in sorted_token_counts:
    if count < FREQUENCY_THRESHOLD:
        break
    # Filter: Jika kata TIDAK ada di STANDARD_STOPWORDS dan TIDAK di WHITELIST
    if word not in STANDARD_STOPWORDS and word not in WHITELIST:
        candidate_additional_stopwords.append(word)

print(f"\nTotal kandidat stopword tambahan (frekuensi > {FREQUENCY_THRESHOLD}): {len(candidate_additional_stopwords)}")
print("--- 20 Kandidat Teratas (Token) ---")
print('\n'.join(candidate_additional_stopwords[:20]))

# =======================================================
# 3. KURASI MANUAL & PEMBENTUKAN FINAL STOPLIST 🚀
# =======================================================

# Kata-kata yang dipilih secara manual (bersifat noise)
CURATED_ADDITIONAL_STOPWORDS = set([
    "baca", "juga", "berita", "lain", "kembali", "lanjut", 
    "sob", "gaes", "yuk", "dunia", "maya", "segera", "download", 
    "jangan", "lupa", "contoh", "terkait", "halaman", "akhir", "artikel",
])

# Gabungkan NLTK Indonesian + Tambahan yang dikurasi
all_stopwords_to_remove = STANDARD_STOPWORDS | CURATED_ADDITIONAL_STOPWORDS | set(candidate_additional_stopwords)

# Kurangi dengan WHITELIST untuk perlindungan
FINAL_STOPLIST = all_stopwords_to_remove - WHITELIST

print(f"\nTotal Stopwords Kandidat (NLTK ID + Tambahan): {len(all_stopwords_to_remove)}")
print(f"Total Kata Kunci di Whitelist: {len(WHITELIST)}")
print(f"✅ Total kata di FINAL STOPLIST: {len(FINAL_STOPLIST)}")
print("-" * 30)

# --- PENYIMPANAN FINAL STOPLIST KE FILE TEKS ---
FINAL_STOPLIST_FILE_PATH = '../data/final_stopwords.txt'

# Pastikan folder '../data' ada sebelum menyimpan
if not os.path.exists(os.path.dirname(FINAL_STOPLIST_FILE_PATH)):
    os.makedirs(os.path.dirname(FINAL_STOPLIST_FILE_PATH))

with open(FINAL_STOPLIST_FILE_PATH, 'w', encoding='utf-8') as f:
    for word in sorted(list(FINAL_STOPLIST)): 
        f.write(word + '\n')

print(f"✅ FINAL STOPLIST berhasil disimpan ke '{FINAL_STOPLIST_FILE_PATH}'.")

# 📢 CETAK ISI FINAL_STOPLIST.TXT
print("\n--- ISI final_stopwords.txt (Contoh 50 Kata Pertama) ---")
try:
    with open(FINAL_STOPLIST_FILE_PATH, 'r', encoding='utf-8') as f:
        lines = [f.readline().strip() for _ in range(50)]
        print('\n'.join(lines))
        print("...")
except FileNotFoundError:
    print(f"Gagal membaca file di: {FINAL_STOPLIST_FILE_PATH}")
print("-----------------------------------")

--- Proses Ekstraksi Stopword Tambahan Dimulai ---
Total token unik yang dianalisis: 309269

Total kandidat stopword tambahan (frekuensi > 25000): 11
--- 20 Kandidat Teratas (Token) ---
baca
salah
bikin
com
memiliki
informasi
seru
menarik
bermain
indogamers
terbaik

Total Stopwords Kandidat (NLTK ID + Tambahan): 780
Total Kata Kunci di Whitelist: 663
✅ Total kata di FINAL STOPLIST: 777
------------------------------
✅ FINAL STOPLIST berhasil disimpan ke '../data/final_stopwords.txt'.

--- ISI final_stopwords.txt (Contoh 50 Kata Pertama) ---
ada
adalah
adanya
adapun
agak
agaknya
agar
akan
akankah
akhir
akhiri
akhirnya
aku
akulah
amat
amatlah
anda
andalah
antar
antara
antaranya
apa
apaan
apabila
apakah
apalagi
apatah
artikel
artinya
asal
asalkan
atas
atau
ataukah
ataupun
awal
awalnya
baca
bagai
bagaikan
bagaimana
bagaimanakah
bagaimanapun
bagi
bagian
bahkan
bahwa
bahwasanya
baik
bakal
...
-----------------------------------


#### Start Preprocessing

In [ ]:
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tqdm.auto import tqdm
import numpy as np

# TESTING
df_article_tes = df_article.head(10).copy()

WHITELIST_FILE_PATH = "../data/whitelist.txt"
FINAL_STOPLIST_PATH = "../data/final_stopwords.txt"
CHECKPOINT_FILE_PATH = "../data/preprocessing_checkpoint.txt"

stemmer = StemmerFactory()

def load_file(file_path):
    """Memuat set kata dari file teks."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        print(f"⚠️ Peringatan: File {file_path} tidak ditemukan. Menggunakan set kosong.")
        return set()
    
WHITELIST = load_words_set(WHITELIST_FILE_PATH)
FINAL_STOPLIST = load_words_set(FINAL_STOPLIST_FILE_PATH)

def preprocess_text(text):
    text = text.lower()

    text = text.replace("-", "")
    text = re.sub(r'[^\w\s]',' ', text)
    text = re.sub

    

In [ ]:
import pandas as pd
import re
import os
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
from tqdm.auto import tqdm 
import numpy as np

# =======================================================
# 📌 SETUP DATA (PENTING!)
# =======================================================

# ASUMSI: df_article (DataFrame asli Anda) sudah dimuat sebelum sel ini.
# --- MODE UJI COBA (Gunakan 10 baris teratas) ---
df_article = df_article.head(10).copy() 

# =======================================================
# PATH DAN INISIALISASI
# =======================================================

WHITELIST_FILE_PATH = '../data/whitelist.txt'
FINAL_STOPLIST_FILE_PATH = '../data/final_stopwords.txt'
CHECKPOINT_FILE_PATH = '../data/preprocessing_checkpoint.txt'

stemmer = StemmerFactory().create_stemmer()

# =======================================================
# 1. FUNGSI MEMUAT FILE
# =======================================================

def load_words_set(file_path):
    """Memuat set kata dari file teks."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            return set(line.strip() for line in f if line.strip())
    except FileNotFoundError:
        print(f"⚠️ Peringatan: File {file_path} tidak ditemukan. Menggunakan set kosong.")
        return set()

# Muat Whitelist dan Final Stoplist
WHITELIST = load_words_set(WHITELIST_FILE_PATH)
FINAL_STOPLIST = load_words_set(FINAL_STOPLIST_FILE_PATH)

print(f"Loaded WHITELIST: {len(WHITELIST)} kata")
print(f"Loaded FINAL STOPLIST: {len(FINAL_STOPLIST)} kata")

# =======================================================
# 2. FUNGSI PEMISAH KATA YANG MENEMPEL (AGRESIF)
# =======================================================

def separate_concatenated_words(text):
    """Memisahkan kata yang menempel akibat tag HTML yang dihapus."""
    if not isinstance(text, str):
        return text
    
    # 1. Pisahkan huruf kecil diikuti huruf besar (camelCase/PascalCase)
    text = re.sub(r'([a-z])([A-Z])', r'\1 \2', text)
    
    # 2. Pisahkan huruf diikuti angka atau angka diikuti huruf
    text = re.sub(r'([a-zA-Z])(\d)', r'\1 \2', text)
    text = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', text)
    
    # 3. Pisahkan huruf besar berurutan diikuti huruf kecil (contoh: HTMLParser)
    text = re.sub(r'([A-Z]+)([A-Z][a-z])', r'\1 \2', text)
    
    # 4. Pemisahan Domain/Kata Gabung Sisa
    text = re.sub(r'([a-z]{5,})(dari|untuk|juga|sebagai|pada)', r'\1 \2', text)

    return text

# =======================================================
# 3. FUNGSI KOREKSI KATA WHITELIST (REJOIN EKSPLISIT)
# =======================================================

def rejoin_whitelisted_words(text, whitelist):
    """
    Menggabungkan kembali kata-kata kunci yang terpisah secara eksplisit.
    """
    if not isinstance(text, str):
        return text
    
    temp_text = text
    
    # Koreksi E3
    if 'e3' in whitelist:
        # Ganti 'e 3' atau 'E 3' menjadi 'e3'
        temp_text = re.sub(r'e\s*3', 'e3', temp_text, flags=re.IGNORECASE) 
    
    # Koreksi PlayStation
    if 'playstation' in whitelist:
        # Ganti 'play station' menjadi 'playstation'
        temp_text = re.sub(r'play\s*station', 'playstation', temp_text, flags=re.IGNORECASE)
        
    # Koreksi PS5
    if 'ps5' in whitelist:
        # Ganti 'ps 5' menjadi 'ps5'
        temp_text = re.sub(r'ps\s*5', 'ps5', temp_text, flags=re.IGNORECASE)
    
    # Koreksi Domain yang Sering Menempel
    temp_text = re.sub(r'detikinet\s*dari', 'detikinetdari', temp_text, flags=re.IGNORECASE)

    return temp_text

# =======================================================
# 4. FUNGSI PREPROCESSING UTAMA (FINAL)
# =======================================================

def preprocess_text(text):
    """
    Pipeline preprocessing teks utama (Case folding, Punctuation, Stopword, Stemming).
    """
    if not isinstance(text, str):
        return ""
        
    # 1. Hapus Noise (HTML)
    text = re.sub(r'<[^>]*>', ' ', text)
    
    # 2. Case Folding
    text = text.lower() # 💡 Teks di sini sudah menjadi lowercase
    
    # 3. Hapus Punctuation (SEBELUM Tokenisasi)
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # 4. Bersihkan multiple spasi jadi satu spasi
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 5. Tokenisasi
    tokens = text.split()
    
    # 6. Stopword Removal (Menggunakan FINAL_STOPLIST)
    tokens_filtered = [word for word in tokens if word not in FINAL_STOPLIST]
    
    # 7. Stemming Selektif dan Penanganan Angka
    stemmed_tokens = []
    for word in tokens_filtered:
        
        # 7a. PROTEKSI AKRONIM HURUF/ANGKA (e.g., ps5, e3)
        # Jika token berisi campuran huruf dan angka, pertahankan UTUH.
        if any(char.isalpha() for char in word) and any(char.isdigit() for char in word):
            stemmed_tokens.append(word)
            continue
            
        # 7b. PROTEKSI ANGKA MURNI (e.g., 2020)
        if word.isdigit():
            stemmed_tokens.append(word)
            continue
            
        # 7c. 💡 KOREKSI: PROTEKSI WHITELIST (Jika token sudah lowercase, ini seharusnya bekerja)
        if word in WHITELIST:
            stemmed_tokens.append(word)
            continue # Tambahkan continue agar tidak jatuh ke stemming di 'else'
        
        # 7d. Stemming jika semua proteksi gagal
        stemmed_tokens.append(stemmer.stem(word))
            
    # 8. Gabungkan kembali menjadi string
    return ' '.join(stemmed_tokens)

# =======================================================
# 5. APLIKASI KE DATAFRAME DENGAN CHECKPOINT & TQDM
# =======================================================

# 5.1. Pemisahan Kata Agresif pada kolom baru
print("\n--- Memisahkan Kata yang Menempel di Kolom 'konten' ---")
df_article['konten_separated_raw'] = df_article['konten'].apply(
    lambda x: separate_concatenated_words(x) if isinstance(x, str) else x
)

# 5.2. Koreksi Balik Berdasarkan Whitelist (Rejoining)
print("Mulai koreksi balik (Rejoining) berdasarkan WHITELIST...")
df_article['konten_separated'] = df_article['konten_separated_raw'].apply(
    lambda x: rejoin_whitelisted_words(x, WHITELIST)
)
print("✅ Koreksi WHITELIST selesai!")


# 5.3. Aplikasi Pipeline Utama dengan Checkpoint
if 'konten_processed' not in df_article.columns:
    df_article['konten_processed'] = np.nan 

total_rows = len(df_article)

# Memuat progress terakhir
start_index = 0
if os.path.exists(CHECKPOINT_FILE_PATH):
    with open(CHECKPOINT_FILE_PATH, 'r') as f:
        try:
            start_index = int(f.read().strip())
        except ValueError:
            start_index = 0 

if start_index >= total_rows:
    print("✅ Preprocessing sudah selesai (sesuai data checkpoint).")
else:
    print(f"\n--- Memulai Proses Preprocessing Kolom 'konten_separated' ---")
    print(f"Melanjutkan dari baris ke-{start_index} dari {total_rows}...")

    for i in tqdm(range(start_index, total_rows), 
                  initial=start_index, 
                  total=total_rows, 
                  desc="Preprocessing Konten"):
        
        if pd.isna(df_article.iloc[i]['konten_processed']): 
            text_to_process = df_article.iloc[i]['konten_separated']
            processed_text = preprocess_text(text_to_process)
            df_article.iloc[i, df_article.columns.get_loc('konten_processed')] = processed_text

            if (i + 1) % 1000 == 0 or i == total_rows - 1:
                with open(CHECKPOINT_FILE_PATH, 'w') as f:
                    f.write(str(i + 1))
    
    print("✅ Preprocessing Selesai.")

# Hapus file checkpoint jika semua baris sudah terproses
if os.path.exists(CHECKPOINT_FILE_PATH) and df_article['konten_processed'].notna().all():
    os.remove(CHECKPOINT_FILE_PATH)
    print("File checkpoint dihapus.")


# =======================================================
# 6. TINJAUAN HASIL & PENYIMPANAN
# =======================================================

if 'konten' in df_article.columns and len(df_article) > 0:
    print("\n--- Contoh Perbandingan Hasil Akhir ---")
    
    indices_to_show = sorted(list(set([0, 1, min(total_rows - 1, 2)])))
    
    for i in indices_to_show:
        if i < total_rows:
            print(f"\n{'='*70}")
            print(f"Baris ke-{i}:")
            print(f"\n1. Konten ASLI:")
            print(f"   {str(df_article.iloc[i]['konten'])[:150]}...")
            print(f"\n2. Konten SETELAH Pemisahan Kata (Dikoreksi Whitelist):")
            print(f"   {str(df_article.iloc[i]['konten_separated'])[:150]}...")
            print(f"\n3. Konten SETELAH Preprocessing:")
            print(f"   {df_article.iloc[i]['konten_processed'][:150]}...")
else:
    print("DataFrame kosong atau kolom 'konten' tidak ada.")

# Simpan DataFrame ke file CSV
df_article.to_csv('../data/dataset_processed_final.csv', index=False)
print(f"\nDataFrame telah disimpan ke '../data/dataset_processed_final.csv'.")

Loaded WHITELIST: 663 kata
Loaded FINAL STOPLIST: 777 kata

--- Memisahkan Kata yang Menempel di Kolom 'konten' ---
Mulai koreksi balik (Rejoining) berdasarkan WHITELIST...
✅ Koreksi WHITELIST selesai!

--- Memulai Proses Preprocessing Kolom 'konten_separated' ---
Melanjutkan dari baris ke-0 dari 10...


Preprocessing Konten:   0%|          | 0/10 [00:00<?, ?it/s]

✅ Preprocessing Selesai.

--- Contoh Perbandingan Hasil Akhir ---

Baris ke-0:

1. Konten ASLI:
   "Setelah evaluasi menyeluruh, Sony memutuskan untuk tidak berpartisipasi di E3 2020. Kami menghormati ESA sebagai organisasi, tapi kami merasa visi E3...

2. Konten SETELAH Pemisahan Kata (Dikoreksi Whitelist):
   "Setelah evaluasi menyeluruh, Sony memutuskan untuk tidak berpartisipasi di e3 2020. Kami menghormati ESA sebagai organisasi, tapi kami merasa visi e3...

3. Konten SETELAH Preprocessing:
   evaluasi sony putus partisipasi e hormat esa organisasi visi e venue fokus juru bicara sony dikutipdetik ine tdari cnet rabu deret judul game hadir pl...

Baris ke-1:

1. Konten ASLI:
   Tahun ini,PlayStation 5bakal diluncurkanSony. Waktunya tak dibeberkan, kabarnya jelang akhir tahun. Tapi menurut developer game yang satu ini, PS5 bak...

2. Konten SETELAH Pemisahan Kata (Dikoreksi Whitelist):
   Tahun ini,playstation 5 bakal diluncurkan Sony. Waktunya tak dibeberkan, kabarnya jelang akhir 